In [1]:
from random import sample
import numpy as np
import pandas as pd 
import os
import sklearn
from sklearn.metrics import confusion_matrix
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (3, 3)
%matplotlib inline

from model import Model
from utils import ImageDataset, show

In [2]:
# Creating labels and splitting data
forest = pd.DataFrame(os.listdir('data/forest'), columns=['number'])
glacier = pd.DataFrame(os.listdir('data/glacier'), columns=['number'])
mountain = pd.DataFrame(os.listdir('data/mountain'), columns=['number'])
sea = pd.DataFrame(os.listdir('data/sea'), columns=['number'])

forest['name_label'] = 'forest'
glacier['name_label'] = 'glacier'
mountain['name_label'] = 'mountain'
sea['name_label'] = 'sea'

name_label_to_num = {
    'forest':0,
    'glacier':1,
    'mountain':2,
    'sea':3,
}

forest['label'] = forest['name_label'].apply(lambda name: name_label_to_num[name])
glacier['label'] = glacier['name_label'].apply(lambda name: name_label_to_num[name])
mountain['label'] = mountain['name_label'].apply(lambda name: name_label_to_num[name])
sea['label'] = sea['name_label'].apply(lambda name: name_label_to_num[name])

i,j = 2300, 2600
train = pd.concat([forest[:i], glacier[:i], mountain[:i], sea[:i]])
valid = pd.concat([forest[i:j], glacier[i:j], mountain[i:j], sea[i:j]])
test = pd.concat([forest[j:], glacier[j:], mountain[j:], sea[j:]])
train

,number,name_label,label
0,10007.jpg,forest,0
1,10010.jpg,forest,0
2,10020.jpg,forest,0
3,10030.jpg,forest,0
4,10037.jpg,forest,0
...,...,...,...
2295,6111.jpg,sea,3
2296,6124.jpg,sea,3
2297,6125.jpg,sea,3
2298,613.jpg,sea,3


In [3]:
# GPU or CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
train_dataset = ImageDataset(train, 'data')
validation_dataset = ImageDataset(valid, 'data')
test_dataset = ImageDataset(test, 'data')

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=0)
validation_dataloader = DataLoader(validation_dataset, batch_size=64, num_workers=0)
test_dataloader = DataLoader(test_dataset, batch_size=len(test), num_workers=0)

In [5]:
#model.load_state_dict(torch.load('model_dict'))
model = Model().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [6]:
train_losses = []
valid_losses = []
num_epochs = 10

for epoch in range(num_epochs):
    # keep-track-of-training-and-validation-loss
    train_loss = 0.0
    valid_loss = 0.0
    
    # training-the-model
    model.train()
    for data, target in train_dataloader:
        # move-tensors-to-GPU 
        data = data.to(device)
        target = target.to(device)
        
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        output = model(data)
        # calculate-the-batch-loss
        loss = criterion(output, target)
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
        # perform-a-ingle-optimization-step (parameter-update)
        optimizer.step()
        # update-training-loss
        train_loss += loss.item() * data.size(0)
        
    # validate-the-model
    model.eval()
    for data, target in validation_dataloader:
        
        data = data.to(device)
        target = target.to(device)
        
        output = model(data)
        
        loss = criterion(output, target)
        
        # update-average-validation-loss 
        valid_loss += loss.item() * data.size(0)
    
    # calculate-average-losses
    train_loss = train_loss/len(train_dataloader.sampler)
    valid_loss = valid_loss/len(validation_dataloader.sampler)
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    save_file = 'saved_models/model_after_'+str(epoch+1)+'_epochs'
    torch.save(model.state_dict(), save_file)
    
        
    # print-training/validation-statistics 
    print(f"Epoch: {epoch+1} \tTraining Loss: {train_loss:.3f} \tValidation Loss: {valid_loss:.3f}")

Epoch: 1 	Training Loss: 11.268 	Validation Loss: 1.387
Epoch: 2 	Training Loss: 1.389 	Validation Loss: 1.389



KeyboardInterrupt

